<a href="https://colab.research.google.com/github/Jang-KyungWuk/Slide_Puzzle_Analysis/blob/master/Sample_Data_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Driver Mount
# Only necessary on goole colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import csv

In [ ]:
# Set flags
save_p_frame = True # Save frames containing puzzle as jpg
fbf_p_tracker_save = True # Save frames containing puzzle and it's tracking as jpg

# Set base directory (directory of this file on colab enviornment)
# When setting base_dir directory should end with \
# For colab '/content/gdrive/My Drive/'
base_dir="/content/gdrive/My Drive/"

# Name of video file to
sample_video="sample_vid.mp4"

# Frame by frame threshold
min_fbf_thres=0.5
max_fbf_thres=0.8
fbf_method=cv.TM_CCOEFF_NORMED

# Piece_threshold
piece_thres=0.95
piece_method=cv.TM_CCOEFF_NORMED

# Puzzle status finder sensitivity
sensitivity=60

# Load puzzle piece templates
puzzle_piece=[]
puzzle_diversity=6 # Last origin has 6 types of puzzles
for puzzle_num in range (puzzle_diversity):
    pieces=[]
    for piece_num in range (2,10): # 2,3,4,5,6,7,8,9 pieces (piece 1 is blank)
        pieces.append(cv.imread(base_dir+"sample_template/%d%d.jpg"%(puzzle_num+1,piece_num)))
    puzzle_piece.append(pieces)

# BGR in list to track pieces
color=[(0,0,255),(0,255,0),(255,0,0),
       (0,200,200),(200,0,200),(200,200,0),
       (200,200,200),(255,255,255)]

In [ ]:
# Program starts from here

# List which will store puzzle number and puzzle status
sample=[]

# Load video with puzzles
cap=cv.VideoCapture(base_dir+sample_video)

# Value used for labeling
capture_cursor=1

# read first frame
ret,frame=cap.read()
template_frame=frame # frame of one step before

# run program until last frame is given
while cap.isOpened():
    ret,frame=cap.read()
    # Stop stream when there's no frame
    if not ret:
        print("Can't recieve frame (or stream ended)")
        break

    # Compare with frame right before
    res=cv.matchTemplate(frame,template_frame,fbf_method)
    # Frames are same on size so only 1 res value returned
    if min_fbf_thres<res[0][0]<max_fbf_thres:

        # Start Analysis if this frame is 'properly' changed
        '''
        High relativity: continuous frame (not needed)
        Proper relativity : puzzle appeared (right after answer pop up)
        Low relativity : answer appeared (only answer shows up first) , puzzle pop off
        '''

        # If flag is given, save this frame
        if save_p_frame == True:
            cv.imwrite(base_dir+"frame%d.jpg"%capture_cursor,frame)

        # List which will save positions of template (puzzle piece)
        piece_pos=[]
        
        # Find puzzle number
        for puzzle_num in range (puzzle_diversity): # Last Origin slide puzzle has 6 type of puzzle
            res=cv.matchTemplate(frame,puzzle_piece[puzzle_num][0],piece_method) # Compare with every 2nd puzzle piece
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
            # If puzzle piece has matching sector in frame
            if max_val>piece_thres:
                piece_pos.append(max_loc)
                # If flag is given draw rectangle at template's position
                if fbf_p_tracker_save==True:
                    BGR,width,height = puzzle_piece[puzzle_num][0].shape[::-1]
                    cv.rectangle(frame,max_loc,(max_loc[0]+width,max_loc[1]+height),color[0],3)
                    cv.putText(frame,"piece"+str(2),(max_loc[0],max_loc[1]-10),0,1,color[0],2)        
                break

        # Track puzzle pieces
        for piece_num in range (1,8): # Last Origin slide puzzle is 3 by 3 puzzle with 8 pieces
            res=cv.matchTemplate(frame,puzzle_piece[puzzle_num][piece_num],piece_method)
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
            piece_pos.append(max_loc)
            # If flag is given draw rectangle at template's position
            if fbf_p_tracker_save==True:
                BGR,width,height = puzzle_piece[puzzle_num][piece_num].shape[::-1]
                cv.rectangle(frame,max_loc,(max_loc[0]+width,max_loc[1]+height),color[piece_num],3)
                cv.putText(frame,"piece"+str(piece_num+2),(max_loc[0],max_loc[1]-10),0,1,color[piece_num],2)  
        # If flag is given, save frame with rectangles
        if fbf_p_tracker_save==True:
            cv.imwrite(base_dir+'frame%d track.jpg'%capture_cursor,frame)
        capture_cursor+=1

        # Use pos to figure puzzle status
        XS=[]
        YS=[]
        for n in range (8):
            x,y=piece_pos[n][0],piece_pos[n][1]
            XS.append(x)
            YS.append(y)

        max_X, min_X = max(XS), min(XS)
        max_Y, min_Y = max(YS), min(YS)

        puzzle=np.zeros(9, dtype=int)
        for n in range(8):
            x=XS[n]
            y=YS[n]
    
            if min_Y<=y<=min_Y+sensitivity:
                pos=0
            elif max_Y-sensitivity<=y<=max_Y:
                pos=6
            else:
                pos=3

            if min_X<=x<=min_X+sensitivity:
                pos+=0
            elif max_X-sensitivity<=x<=max_X:
                pos+=2
            else:
                pos+=1
            puzzle[pos]+=(n+2)

        puzzle=list(puzzle)
        puzzle[puzzle.index(0)]='B'
        
        pattern=str(puzzle[0])
        for puzzle_cursor in range (1,9):
            pattern+=str(puzzle[puzzle_cursor])

        # save as puzzle number, puzzle pattern
        sample.append([puzzle_num+1,pattern])
        
    # Current frame will become past frame of future frame
    template_frame=frame

Can't recieve frame (or stream ended)


In [ ]:
print("From video sample, %s sample puzzles were extracted"%(capture_cursor-1))

From video sample, 500 sample puzzles were extracted


In [ ]:
# Save result as csv
with open (base_dir+"samples.csv",'w',newline='') as myfile:
    wr=csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for cursor in range (len(sample)):
        wr.writerow(sample[cursor])